In [1]:
%%javascript
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('=,=', 'jupyter-notebook:restart-kernel-and-run-all-cells');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('F2,F2', 'jupyter-notebook:restart-kernel');
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('F1,F1', 'jupyter-notebook:run-all-cells-above');

<IPython.core.display.Javascript object>

In [2]:
from glob import glob
import os
import signal
import sys

from IPython.display import clear_output, display
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import torch
from tqdm import tqdm#_notebook

floorplan_vectorization_path = '/home/apankov/dev.vectorization'
sys.path.append(floorplan_vectorization_path)
from vectran.optimization.primitives.line_tensor import LineTensor
from vectran.optimization.primitives.quadratic_bezier_tensor import QuadraticBezierTensor
from vectran.optimization.optimizer.adam import Adam

from vectran.data.graphics.graphics import VectorImage
from vectran.renderers.cairo import render

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

dtype = torch.float32
device = torch.device('cuda')
monitoring = True

In [3]:
patch_width = 64
patch_height = 64

## Load data

In [4]:
filename = glob('/home/ovoinov/remotes/root.zhores/gpfs/data/gpfs0/3ddl/vectorization/datasets/svg_datasets/whole_images/abc/test/\
00050012*.svg')[0]
vector_image = VectorImage.from_svg(filename)
# Image.fromarray(vector_image.render(render))

In [5]:
vector_patches = vector_image.split_to_patches([patch_width, patch_height])
vector_patches = vector_patches[5:9, 3].ravel()
raster_patches = [patch.render(render) for patch in vector_patches]
patches_n = len(raster_patches)
# Image.fromarray(np.vstack(raster_patches))

## Optimize

In [6]:
# primitive_type = 'lines'
primitive_type = 'curves'
primitives_n = 30

if primitive_type == 'lines':
    prim_ten = LineTensor(
        np.random.rand(patches_n, 2, primitives_n) * [[patch_width], [patch_height]],
        np.random.rand(patches_n, 2, primitives_n) * [[patch_width], [patch_height]],
        np.random.rand(patches_n, 1, primitives_n) + 1,
        dtype=dtype, device=device)
elif primitive_type == 'curves':
    prim_ten = QuadraticBezierTensor(
        np.random.rand(patches_n, 2, primitives_n) * [[patch_width], [patch_height]],
        np.random.rand(patches_n, 2, primitives_n) * [[patch_width], [patch_height]],
        np.random.rand(patches_n, 2, primitives_n) * [[patch_width], [patch_height]],
        np.random.rand(patches_n, 1, primitives_n) + 1,
        dtype=dtype, device=device)
    
q_raster = 1 - torch.as_tensor(raster_patches).type(dtype) / 255

aligner = Adam(prim_ten, q_raster, loglevel='error', lr=1)
if monitoring:
    fig = aligner.prepare_visualization(store_plots=True)
plt.close()

In [ ]:
iters_n = 100000
draw_each = 10

its_time_to_stop = [False]
def plotting_sigint(*args):
    its_time_to_stop[0] = True

for i in tqdm(range(iters_n)):
    try:
        aligner.step(i, draw_visualization=(monitoring and ((i % draw_each) == 0)))
    
    except KeyboardInterrupt:
        its_time_to_stop[0] = True
    sigint = signal.signal(signal.SIGINT, plotting_sigint)
    
    if monitoring and ((i % draw_each) == 0):
        clear_output(wait=True)
        display(fig)
    
    signal.signal(signal.SIGINT, sigint)
    if its_time_to_stop[0]:
        break

In [ ]:
assert False

In [ ]:
aligner.save_plots('/home/ovoinov/Downloads/primopt_validate_eccv.30_curves.mp4', fps=10)